In [3]:
from datasets import load_dataset
# The 'cnn_dailymail' dataset is pre-built and hosted, so no data_files should be necessary.
dataset = load_dataset(
    "cnn_dailymail",
    name="3.0.0", 
    split="train" 
)

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Using HuggingFace Transformers library BART which is a pre trained model to summarize an article from the CNN dailymail dataset. test the model using ROUGE which is widely used for evaluating text summarization

In [5]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

# Load pre-trained BART model and tokenizer
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Function to summarize text
def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding=True)

    # Generate the summary using the model
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, min_length=50, max_length=150, early_stopping=True)

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


article = dataset[0]["article"]  # Get the first article
summary = summarize(article)

print("Original Article:")
print(article)
print("\nGenerated Summary:")
print(summary)

Original Article:
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office cha

In [8]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00


In [10]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=be31b262b68316d6ffe7e0fba81f573759d9b35aae60428070f4658e30f769f1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [12]:
from evaluate import load

# Load the ROUGE metric
rouge = load("rouge")

# Evaluate the generated summary against the reference summary
references = [dataset[0]["highlights"]]  # Ground truth summary
generated = [summary]

# Calculate ROUGE score
result = rouge.compute(predictions=generated, references=references)
print(result)


{'rouge1': 0.6136363636363635, 'rouge2': 0.372093023255814, 'rougeL': 0.5454545454545454, 'rougeLsum': 0.5681818181818182}
